# 

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import copy
import math
import json
from scripts import utils
import importlib
import matplotlib.pyplot as plt
MAIN_DIR = "C:/Users/nourisa/Downloads/testProjs/proteomics_MSC"
IMPUT_FILE = os.path.join(MAIN_DIR,'statistical_analysis_2/ProteinAbundance_tables/ProtAbundance__Norm_n_Imp_SVD.csv') # dir for the imputated df 
SIG_FILE = os.path.join(MAIN_DIR, 'statistical_analysis_2/DiffExp_tables/TopTable_TimeCourse_SVD.csv') # dir for the sig analysis
OUTPUT_DIR = os.path.join(MAIN_DIR,'results')
sys.path.insert(0,MAIN_DIR)
specs = dict(
    o_df_dir = os.path.join(MAIN_DIR,'data','original_omics.xlsx'),
    df_dir = os.path.join(MAIN_DIR,'data','primary_omics.csv'),
    time = [1,2,3,4,7,8,9,10,11,14,21],
    p_name = 'Entry',  # The column name in the original data to be used as protein name
    c_tag = 'ctr_',
    s_tag = 'mg_',
    c_func = lambda t: 'ctr_' + str(t),  # The tag used in the tailored dataframe for control
    s_func = lambda t: 'mg_' + str(t),  # The tag used in the tailored dataframe for sample
    o_c_func = lambda t: 'LogLFQ intensity ' + str(t) + '_0',  # Func to extract the control data from the original database 
    o_s_func = lambda t: 'LogLFQ intensity ' + str(t) + '_1',  # Func to extract the sample data from the original database
)

# Edit the original df to create primary df 
Read the original proteomics data, process it by:
renaming the missing symbols
changing column names (to make it easier to follow)
removing rows with all zeros

In [ ]:
importlib.reload(utils)
## read the original data 
o_df = pd.read_excel(specs['o_df_dir'])
# process the data
o_df_m = utils.rename_missing_symbols(o_df,**specs); 
df = utils.tailor_names(o_df_m, **specs)
df = utils.remove_zeros(df, **specs)
df.to_csv(specs['df_dir'])

In [ ]:
# nonStrIndices = [type(item) != str for item in df['Gene']]
# df.loc[nonStrIndices,'Entry']

# Post statistical analysis
- Read imputed and DE results, and sort them based on protein names
- 

In [2]:
# load the imputed df
df_imput = pd.read_csv(IMPUT_FILE)
df_imput.rename(columns={'Unnamed: 0':'Protein'},inplace=True)
# read the sig results
df_sig = pd.read_csv(SIG_FILE)
# sort the data to make them comparable
df_imput.sort_values('Protein', inplace=True)
df_imput.reset_index(drop=True, inplace=True)
df_sig.sort_values('Protein', inplace=True)
df_sig.reset_index(drop=True, inplace=True)
prots_names = df_sig['Protein'].values
# calculate fold change: log(mg)-log(ctr)
df_diff = pd.DataFrame() #columns: ['Protein','1','2', etc (time points)]
df_diff['Protein'] = df_imput['Protein']
for t in specs['time']:
    ctr = f'ctr_{t}'
    mg = f'mg_{t}'
    df_diff[t] = (df_imput.loc[:,mg]-df_imput.loc[:,ctr]).values
# select 
DE_t = .05
fold_change_t = 1.5
df_flags = pd.DataFrame()#columns=['Protein','Sig','FC_1','FC_2',etc,'Overall']
df_flags['Protein'] = df_imput['Protein']
df_flags['Sig'] = df_sig['P.Value']<DE_t
for t in specs['time']:
    tag = f'FC_{t}'
    df_flags[tag] = abs(df_diff[t]) > np.log2(fold_change_t)
df_flags['DE'] = df_flags['Sig']* np.any(df_flags.iloc[0:,2:].values,axis=1)
# output the DE proteins to a file
DE_protnames = list(df_flags.loc[df_flags['DE'],'Protein'].values)
DE_protnames_str = ''
for gene in DE_protnames:
    DE_protnames_str+=gene + ','
with open(os.path.join(MAIN_DIR,'results/postprocess','DE_protnames.txt'),'w') as f:
    print({'DE_protnames':DE_protnames, 'DE_protnames_str':DE_protnames_str}, file=f)

In [ ]:
# volcanic plot: -log10(p values) vs log2(foldchange)
# import matplotlib.pyplot as plt
# sig_cut_off = -np.log10(.05)
# FC_cut_off = np.log2(1.5)
# # calculate log10(p-values)
# negLog10pv = -np.log10(list(df_SA_sort['P.Value'].values))
# # calculate log2 fold change (log(mg)-log(ctr)). The average of different time was taken into account 
# df_diff = pd.DataFrame()
# df_diff['Entry'] = df_imput['Entry']
# for t in specs['time']:
#     ctr = f'ctr_{t}'
#     mg = f'mg_{t}'
#     df_diff[t] = (df_imput_sort.loc[:,mg]-df_imput_sort.loc[:,ctr]).values

# # flags: to find those passing the thresholds
# sig_flags = negLog10pv>=sig_cut_off
# FC_flags = abs(df_diff.iloc[0:,1:].values)>FC_cut_off 
# flags = FC_flags * sig_flags[:,None] # n_genes*n_days
# # select those with at least one FC>threshold across different time points
# flags_a = [np.any(row) for row in flags]
# sig_indices = [i for i in range(len(flags_a)) if flags_a[i]]
# sigs_prot = df_imput_sort.loc[sig_indices,'Entry'].values
# print('number of sig proteins: ', len(sig_prots))
# # extract df with sig diff
# df_sig = df_imput_sort.loc[df_imput_sort['Entry'].isin(sigs_prots)]
# df_sig.to_csv('results/statAnalysis/df_sig.csv',index=False)
# # create df for the selected proteins
# df_chosen = df_diff.loc[df_diff['Entry'].isin(sig_prots),:]
# df_chosen.columns=['Protein']+[f'FC (Day {t})' for t in specs['time']]
# df_chosen.loc[:,'P-Value'] = df_SA_sort.loc[df_SA_sort['Entry'].isin(sig_prots),'P.Value'].values
# df_chosen = df_chosen.sort_values('P-Value')

In [ ]:
# volcanic plot: -log10(p values) vs log2(foldchange)
class Plot:
    linewidth = 2
    max_diff = np.max(df_diff.iloc[0:,1:].values)
    fold_change_line = np.log2(fold_change_t)
    @staticmethod
    def mark_p_value(ax,sig_cut_off):
        """
        Markline for the sig
        """
        line_color = 'grey'
        dx = ax.get_xlim()
        ax.axhline(sig_cut_off,color='grey', linestyle='-.')
    @staticmethod
    def mark_FC(ax,FC_cut_off):
        """
        Markline for fold change
        """
        line_color = 'grey'
        FC_cut_offs = [-FC_cut_off,FC_cut_off]
        for FC_cut_off in FC_cut_offs:
            ax.axvline(FC_cut_off,color=line_color, linestyle='--',linewidth=Plot.linewidth)
    @staticmethod
    def scatter(ax,log2FC,negLog10pv,flags):
        ax.scatter(log2FC,negLog10pv,
                   color=['red' if flag else 'black' for flag in flags],
#                    alpha=[1 if flag else .5 for flag in flags]
                   alpha = .7,
                   linewidths=.5,
                   edgecolors='cyan'
                  )
    def print_names(ax, log2FC, flag):
        fontsize = 7
        log2FC_s = log2FC[flag].values
        gene_names_s = df_flags.loc[flag, 'Protein']
        sign_flag = log2FC_s>=0
        gene_names_right = gene_names_s[sign_flag]
        gene_names_negative = gene_names_s[~sign_flag]
        for i, name in enumerate(gene_names_right):
            ax.annotate(name,(1.37,3.4-.3*i),fontsize=fontsize)
        for i, name in enumerate(gene_names_negative):
            ax.annotate(name,(-1.75,3.4-.3*i),fontsize=fontsize)
#         ax.annotate(text_right, (1.1,1),fontsize=fontsize)
    @staticmethod
    def postprocess(ax,time):
        ax.set_xlabel('Log₂ (fold-change)')
        ax.set_ylabel('- Log₁₀ (p-value)')
        ax.set_xlim([-1.3*Plot.max_diff, 1.3*Plot.max_diff])
        ax.set_title(f'Day {time}')
        ax.set_xticks([-Plot.max_diff,-Plot.fold_change_line,int(0),Plot.fold_change_line, Plot.max_diff])
        ax.set_xticklabels([-round(Plot.max_diff,2),-round(Plot.fold_change_line,2),int(0),round(Plot.fold_change_line,2),round(Plot.max_diff,2)])
def plot_multiple(axes, days, cols):
    for i, time in enumerate(days):
        ax = axes[int(i/cols)][i%cols]
        log2FC = df_diff[time]
        negLog10pv = -np.log10(df_sig['P.Value'].values)
        DE_flags_time = (df_flags['Sig'] * df_flags[f'FC_{time}']).values 
        Plot.scatter(ax,log2FC,negLog10pv, DE_flags_time)
        Plot.mark_p_value(ax,-np.log10(sig_t))
        Plot.mark_FC(ax,np.log2(fold_change_t))
        Plot.postprocess(ax,time)
        Plot.print_names(ax,log2FC,DE_flags_time)
def plot_selected():
    days = [1,3,10,14]
    rows = 2
    cols = 2
    fig, axes = plt.subplots(rows, cols, tight_layout=True, figsize=(cols*4, rows*3))
    plot_multiple(axes, days, cols)
    fig.savefig(os.path.join(OUTPUT_DIR,'statAnalysis','volcanic_selected.pdf'))
    fig.savefig(os.path.join(OUTPUT_DIR,'statAnalysis','volcanic_selected.png'),facecolor='white',dpi=300)

def plot_all():
    rows = 6
    cols = 2
    fig, axes = plt.subplots(rows, cols, tight_layout=True, figsize=(cols*4, rows*3))
    plot_multiple(axes, specs['time'], cols)
    fig.savefig(os.path.join(OUTPUT_DIR,'statAnalysis','volcanic_all.pdf'))
    fig.savefig(os.path.join(OUTPUT_DIR,'statAnalysis','volcanic_all.png'),facecolor='white',dpi=300)
# plot_all()
plot_selected()

In [ ]:
# plot table of FC and pvalues: TODO:this needs work
from matplotlib.ticker import FormatStrFormatter
fig, ax = plt.subplots(1, 1, tight_layout=True, figsize=(4, 3))
fig.patch.set_visible(False)
ax.axis('off')
ax.axis('tight')
df_chosen = df_chosen.iloc[0:10,:]
df_chosen_r = df_chosen.round(4)
table= plt.table(cellText=df_chosen_r.values, colLabels=df_chosen_r.columns, loc='center')
from matplotlib.font_manager import FontProperties
for (row, col), cell in table.get_celld().items():
    if (row == 0):
        cell.set_text_props(fontproperties=FontProperties(weight='bold'))
fig.savefig(os.path.join(OUTPUT_DIR,'table.pdf'))

# Enrichment analysis

In [3]:
# convert differnt gene annotation to one another. protname (UniProt name), genename,
# geneID (entrenz gene id). https://www.uniprot.org/id-mapping/ 
from scripts import annotaion_map
import json
importlib.reload(annotaion_map)
with open(os.path.join(MAIN_DIR, 'results/postprocess','DE_protnames.txt')) as f:
    DE_protnames = eval(f.read())['DE_protnames']

map_protname_geneID = annotaion_map.protname_geneID(DE_protnames)
with open(os.path.join(MAIN_DIR,'results/postprocess','map_protname_geneID.json'),'w') as f:
    f.write(json.dumps({'map':map_protname_geneID}))
# map protname to genename according to string: because it is different using the website
string_map = pd.read_csv(os.path.join(MAIN_DIR,'results/enrichment_analysis/string/string_mapping.tsv'), sep='\t',index_col=False)
map_protname_genename = {key:value for key,value in zip(string_map['queryItem'], string_map['preferredName'])}
with open(os.path.join(MAIN_DIR,'results/postprocess','map_protname_genename.json'),'w') as f:
    f.write(json.dumps({'map':map_protname_genename}))
# rest
map_genename_protname = {value:key for value, key in zip(map_protname_genename.values(),map_protname_genename.keys())}
with open(os.path.join(MAIN_DIR,'results/postprocess','map_genename_protname.json'),'w') as f:
    f.write(json.dumps({'map':map_genename_protname}))
map_genename_geneID = {key:map_protname_geneID[value] for key, value in map_genename_protname.items()}
with open(os.path.join(MAIN_DIR,'results/postprocess','map_genename_geneID.json'),'w') as f:
    f.write(json.dumps({'map':map_genename_geneID}))
# write geneIDs to a file
with open(os.path.join(MAIN_DIR, 'results/postprocess','DE_geneIDs.json'),'w') as f:
    f.write(json.dumps({'geneIDs':list(map_protname_geneID.values())}))

Fetched: 48 / 48


In [11]:
# format strings enrichment data  for R    
def reireive_string_data_F(file_type):
    '''
        Reads string EA data such as enrichment.Function
    '''
    FILE =  os.path.join(os.path.join(MAIN_DIR, 'results/enrichment_analysis/string'),file_type)
    data = pd.read_csv(FILE,sep='\t',index_col=False)
    # remove some columns
    data.drop('matching proteins in your network (IDs)',axis=1,inplace=True) 
    data.drop('background gene count',axis=1,inplace=True) 
    data.columns = ['ID','Description', 'Count', 'Strength','p.adjust','genenames']
    # set gene ratio is in the format of x/total
    GeneRatio = []
    for item in data['Count']:
        GeneRatio.append(str(item)+'/'+str(len(DE_protnames)))
    data.loc[:,'GeneRatio'] = GeneRatio
    # not useful but required as input to the graph
    data['pvalue'] = data['p.adjust'] 
    data['qvalue'] = data['pvalue'] 
    data['BgRatio'] = data['GeneRatio'] #TODO: fix this
    # change the gene names to gene ID
    geneIDs_list = []
    for genenames in data['genenames']:
        geneIDs = [map_genename_geneID[genename] for genename in genenames.split(',')]
        geneIDs_str = '/'.join(geneIDs)
        geneIDs_list.append(geneIDs_str)
        
    data.loc[:,'geneID'] = geneIDs_list
    data.drop('genenames', axis=1, inplace=True)
    data.drop('Strength', axis=1, inplace=True) #TODO: fix this
    return data
data = reireive_string_data_F('enrichment.Process.tsv')
data.to_csv(os.path.join(MAIN_DIR,'results/postprocess','enrichment_process.csv'))

In [ ]:

df_clusters = pd.read_csv(os.path.join(MAIN_DIR,'string','string_MCL_clusters.tsv'),sep='\t')
df_clusters

# Noncategorized 

In [ ]:
# plot the time series of sig prots
utils.plot_time_series(df, prots=sig_prots, c_tag='ctr_', s_tag='mg_', p_ID='Entry', time=specs['time'], ee=0.5)

In [ ]:
# visualize the sig proteins for original and imputed versions
from scripts import utils
candids = sig_prots[0:10]
utils.plot_time_series_mutual(df1=df,df2=df_imput, prots=candids, )

plot time serties of Moreneo's sig prots and check which ones are detected in our analysis

In [ ]:
from scripts import utils
importlib.reload(utils)
moreneo_sigs = ['SDF4','NUFIP2','BUB3','HMGA2','KIF5B','DYNC1I2','DCTN1','S100A10','SYNPO2','MARCKS','THY1','HUWE1','PRPF8','RPS18','GCN1L1','RPL14','RBM8A','RPL23A','CANX','DLST','PDHB','ALDOC','APOA1','GGH','UAP1','ENO2','BAG3','HNRNPAB','BASP1','THRAP3','LRPPRC','CSE1L','PABPC4']
utils.plot_time_series(df, prots=moreneo_sigs[0:len(moreneo_sigs)], c_tag='ctr_', s_tag='mg_', p_ID='Gene', time=specs['time'], ee=0.5)

In [ ]:
# check the overlap between Moreneo's sig prots and this study
df_overlap = df_sig.loc[df_sig['Gene'].isin(moreneo_sigs)]
prots_overlap = df_overlap['Entry'].values
print(prots_overlap)
# utils.plot_time_series(df, prots=prots_overlap, c_tag='ctr_', s_tag='mg_', p_ID='Gene', time=specs['time'], ee=0.5)
# print(prots_overlap)
df_sig.head()

In [ ]:
# imputing code
import copy
def KNNImputer(df, n_neighbors=2): #TODO: n neighbors should be evaluated
    from sklearn import impute
    df_copy = copy.deepcopy(df)
    data = df[[f'ctr_{i}' for i in time]+[f'mg_{i}' for i in time]].values
    imputer = impute.KNNImputer(n_neighbors=n_neighbors)
    new_data = imputer.fit_transform(data)
    df_copy[[f'ctr_{i}' for i in time]+[f'mg_{i}' for i in time]] = new_data
    return df_copy
df_imput = KNNImputer(df) #TODO: this can be experimented
for gene in df_imput['Entry'].values:
    if df_imput.loc[df_imput['Entry']==gene].isna().values.any():
        print('still missing values: ', gene)